<a href="https://colab.research.google.com/github/TrvKhanh/Vietnamese-sentiment/blob/main/vietnamese_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q
!pip install underthesea -q
!pip install --upgrade transformers -q
!pip install evaluate -q
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# **PROCESSING DATA**

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

import torch
import re
import evaluate
import numpy as np
from underthesea import text_normalize, word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ds = load_dataset("sepidmnorozy/Vietnamese_sentiment")
x = ds['train']['text']
def normalize_text(text):
    text = text.lower()
    pattern = r'[^\w\s]'
    new_text = re.sub(pattern, "", text)
    text_clear = text_normalize(new_text)
    return text_clear



file_path = "/content/drive/MyDrive/vietnamese-stopwords.txt."

def load_stopword(path=file_path):
  try:
    with open(path, 'r', encoding='utf-8') as file:
        stop_words = [line.strip() for line in file]
  except FileNotFoundError:
    print("Error: vietnamese-stopwords.txt not found.")
    stop_words = []
  return stop_words

def remove_stopwords(text, stopwords):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stopwords]
    return " ".join(filtered_words)

def preprocess_text_dataset(dataset):
    dataset = dataset.map(lambda x: {"text": normalize_text(x["text"])})
    dataset = dataset.map(lambda x: {"text": remove_stopwords(x["text"], load_stopword())})
    return dataset

ds = preprocess_text_dataset(ds)

Using the latest cached version of the dataset since sepidmnorozy/Vietnamese_sentiment couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/sepidmnorozy___vietnamese_sentiment/default/0.0.0/8e816cff499ad7d8af3763d7167ac48512f19c15 (last modified on Wed Mar 12 16:21:23 2025).


In [ ]:
for i in range(30):
  print(ds['train'][i])

{'label': 1, 'text': 'android nhập liệu bằng lời nói rất chuẩn nhanh nhiều ngôn ngữ cùng lúc'}
{'label': 0, 'text': 'sau một thời gian sử dụng aw mình thấy một số điểm chưa hài lòng không đi bơi được ngày nào mình cũng đi tập gym song đi bơi luôn nhưng phải tháo ra rất bực mình do để trong tủ đồ công cộng sợ bị móc chộm quá phụ thuộc vào iphone tính năng nghe gọi nhắn tin hầu như không có tác dụng khi mà iphone kè kè bên mìnhtrừ lúc vào toilet đi ị lúc đó có điện thoại nghe gọi là ổn nhất ở vn ko sài được apple pay tính năng này có cũng như ko mặt aw luôn tối thui như cái đồng hồ rụng kim ps nhình chung dùng cho biết thôi sau một thời gian 34 tháng sẽ thấy nhàm chán và ko sử dụng được nhiều tối về lại phải cắm thêm sạc quên là hôm sau khỏi dùng'}
{'label': 0, 'text': 'phét tôi dùng con 3 gs nguyên bản vào fb lag lòi cả con mắt ra máy chả dám cài cái gì cả chỉ để nghe gọi'}
{'label': 0, 'text': 'bb này ra giá 335 tr có lẽ mới cảm thấy shock chứ giá này thì shoot luôn'}
{'label': 0, 'tex

# **Train Teacher Model**

In [ ]:
base_model = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(base_model)

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}
model = AutoModelForSequenceClassification.from_pretrained(base_model,
                                                           num_labels=2, id2label=id2label, label2id=label2id)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

# **Freeze base model**

In [ ]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

for name, param in model.base_model.named_parameters():
   param.requires_grad = False
   if "classifier" in name:
     param.requires_grad = True

for name, param in model.named_parameters():
   print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight True
roberta.embeddings.position_embeddings.weight True
roberta.embeddings.token_type_embeddings.weight True
roberta.embeddings.LayerNorm.weight True
roberta.embeddings.LayerNorm.bias True
roberta.encoder.layer.0.attention.self.query.weight True
roberta.encoder.layer.0.attention.self.query.bias True
roberta.encoder.layer.0.attention.self.key.weight True
roberta.encoder.layer.0.attention.self.key.bias True
roberta.encoder.layer.0.attention.self.value.weight True
roberta.encoder.layer.0.attention.self.value.bias True
roberta.encoder.layer.0.attention.output.dense.weight True
roberta.encoder.layer.0.attention.output.dense.bias True
roberta.encoder.layer.0.attention.output.LayerNorm.weight True
roberta.encoder.layer.0.attention.output.LayerNorm.bias True
roberta.encoder.layer.0.intermediate.dense.weight True
roberta.encoder.layer.0.intermediate.dense.bias True
roberta.encoder.layer.0.output.dense.weight True
roberta.encoder.layer.0.output.dense.bia

# **Preprocess text**

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
    )

tokenized_data = ds.map(preprocess_function, batched=True)
# Tạo data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

# **Train Model**

In [ ]:
# hyperparameters
lr = 2e-5
batch_size = 8
num_epochs = 20

training_args = TrainingArguments(
    output_dir= "/content/drive/MyDrive/trained_model_vietnamese_sentiment",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id="khanhTran03/fine-tune-phobert-phone-sentiment"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.669900,0.642427,0.666000,0.771000
2,0.640700,0.611427,0.701000,0.797000
3,0.611700,0.584573,0.720000,0.804000
4,0.599400,0.564983,0.733000,0.814000
5,0.583600,0.551102,0.736000,0.818000
6,0.566500,0.544447,0.726000,0.821000
7,0.564500,0.531707,0.736000,0.824000
8,0.553600,0.526977,0.740000,0.824000
9,0.556000,0.521332,0.740000,0.825000
10,0.546300,0.517438,0.737000,0.829000


TrainOutput(global_step=5960, training_loss=0.5637177742567638, metrics={'train_runtime': 483.947, 'train_samples_per_second': 98.523, 'train_steps_per_second': 12.315, 'total_flos': 1856552717722560.0, 'train_loss': 0.5637177742567638, 'epoch': 20.0})

# **Save model**

In [ ]:
model.save_pretrained("trained_model_vietnamese_sentiment")
tokenizer.save_pretrained("trained_model_vietnamese_sentiment")

('trained_model_vietnamese_sentiment/tokenizer_config.json',
 'trained_model_vietnamese_sentiment/special_tokens_map.json',
 'trained_model_vietnamese_sentiment/vocab.txt',
 'trained_model_vietnamese_sentiment/bpe.codes',
 'trained_model_vietnamese_sentiment/added_tokens.json')

# **Apply Model to Validation Dataset**

In [ ]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.716, 'AUC': 0.797}


# **Inference**

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load model, tokenizer
model_path = "trained_model_vietnamese_sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# inference
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
import torch

# Văn bản cần dự đoán
text = "điện thoại rất đẹp"

# Tokenize văn bản
inputs = tokenizer(text, return_tensors="pt")

# Chuyển dữ liệu lên GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Dự đoán
with torch.no_grad():
    outputs = model(**inputs)

# Lấy nhãn dự đoán
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_label = torch.argmax(predictions).item()

print(f"Predicted label: {predicted_label}")

Predicted label: 1


In [ ]:
trainer.push_to_hub("Upload fine-tuned model")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1741797582.624789b428f3.626.1:   0%|          | 0.00/17.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/khanhTran03/fine-tune-phobert-phone-sentiment/commit/21ce772db39354d07b568781403814f0bc68055a', commit_message='Upload fine-tuned model', commit_description='', oid='21ce772db39354d07b568781403814f0bc68055a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/khanhTran03/fine-tune-phobert-phone-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='khanhTran03/fine-tune-phobert-phone-sentiment'), pr_revision=None, pr_num=None)